In [1]:
import random
from datetime import datetime, date, time
from selenium import webdriver
from time import sleep
from functools import partial
import pandas as pd
from glob import glob
from os import path
import re
from hashlib import md5
from threading import Thread, Lock
from bs4 import BeautifulSoup
import requests

In [2]:
def random_sleep(a = 0, b = 0):
    pass

In [3]:
def save_data():
    global data
    global data_lock
    data_lock.acquire()
    
    df = pd.DataFrame(data)
    df.to_csv(path.join('data', r'avito_data{}.csv').format(str(datetime.now()).replace(':', '.')))
    
    for key in data.keys():
        data[key] = []
        
    data_lock.release()
    
def add_data(new_data):
    global data
    global data_lock
    data_lock.acquire()
    for key, item in normalize(new_data).items():
        data[key].append(item)
    data_lock.release()

In [4]:
def pop_account():
    global accounts
    global accounts_lock
    accounts_lock.acquire()
    while len(accounts) == 0:
        sleep(1)
    acc = accounts.pop(0)
    accounts_lock.release()
    print('start to parse account:', acc)
    return acc

In [5]:
def get_proxy_gen(file='proxy.txt'):
    while True:
        with open(file, 'r') as f:
            for proxy in f:
                yield proxy.strip()

def get_proxy():
    global proxer
    global proxer_lock
    proxer_lock.acquire()
    proxy = next(proxer)
    proxer_lock.release()
    return proxy

def get_html_page(url):
    while True:
        try:
            proxies = {
                'https': 'http://' + get_proxy(),
            }
            req = requests.get(url, proxies=proxies)
            if (not req.ok) or (req.text.find('Доступ временно заблокирован') > 0) or\
                    (req.text.find('Доступ с вашего IP-адреса временно ограничен') > 0):
                continue
            return req
        except Exception as e:
#             print(e)
            continue
    

In [6]:
def get_new_browser(browser = None):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument(r'--proxy-server=http://' + get_proxy())
    if browser is not None:
        try:
            browser.quit()
            
        except Exception as e:
#             print(e)
            pass
    return webdriver.Chrome(options=chrome_options)

In [7]:
def get_url(browser, url):
    browser.get(url)
    while browser.title in ('Доступ временно заблокирован', 'www.avito.ru', 'Доступ с вашего IP-адреса временно ограничен'):
        browser = get_new_browser(browser)
        browser.get(url)
    return browser

In [8]:
def get_visited_products(path=None):
    from os import path as path_
    if path is None:
        path = path_.join('data', 'visited_products.txt')
    try:
        with open(path, 'r') as file:
            return set([product.strip() for product in file])
    except IOError as e:
        print('error in get_visited_products function: ', e)
        return set()
def get_visited_accounts(directory='data', formatted='csv'):    
    try:
        urls = set()
        for file_name in glob(path.join(directory, '*.' + formatted)):
            df = pd.read_csv(file_name)
            for long_url in df.url:
                url = re.match(r"https://www.avito.ru/user/(.*)/profile", long_url).group(0)
                urls.add(url)
        return urls
    except Exception as e:
        print('error in get_visited_accounts function', e)
        return set()
def visit_product(url, path=None):
    from os import path as path_
    path = path or path_.join('data', 'visited_products.txt')

    global visited_products
    global visited_products_lock
    
    try:
        visited_products.add(url)
        with open(path, 'a') as file:
            print(url, file=file)
    except IOError as e:
        print('error in visit_product function: ', e)

In [9]:
def good_url(url):
    '''Функция чекает список новых урлов на акки, отсеивает посещённые, оставляет нужные по хешу и тд и тп'''
    global visited_accounts
    global HASH_MOD
    
    return not (url in visited_accounts) and url.startswith('https://www.avito.ru/user/') and \
            int(md5(url.encode('utf-8')).hexdigest(), base=16) % 6 in HASH_MOD

In [10]:
def get_acc_from_product(args):# args = [i1, i2, ..., ik] in products array
    global products
    global accounts
    global visited_accounts
    global visited_accounts_lock
    
    while True:
        for i in args:
            if products[i] is not None:
                req = get_html_page(products[i])
                start = req.text.find('/user/')
                finish = req.text.find('/profile')
                if start >= 0 and finish >= 0:
                    url = r'https://www.avito.ru' + req.text[start:finish+len('/profile')]
                    visited_accounts_lock.acquire()
                    if good_url(url):
                        print('got account:', url)
                        accounts.append(url)
                        visited_accounts.add(url)
                    visited_accounts_lock.release()
                products[i] = None
        sleep(5)

In [11]:
def get_accounts_from_page(url):
    #данные о проверенных товарах
    global visited_products
    global products

    req = get_html_page(url)
    soup = BeautifulSoup(req.text, 'html.parser')
    elements = soup.findAll('a', class_='item-description-title-link')
    urls = list(['https://www.avito.ru' + i.get_attribute_list('href')[0] for i in elements])
    buf = []
    for i in range(len(urls)):
        product_url = urls[i]
        if not (product_url in visited_products):
            visit_product(product_url)
            for i in range(54):
                if products[i] is None:
                    products[i] = product_url
                    break
            else:
                buf.append(product_url)
    j = 0
    while j < len(buf):
        for i in range(54):
            if products[i] is None:
                products[i] = product_url
                j += 1
                break
        else:
            sleep(5)

In [ ]:
def search(url, start_page = 1):
    '''url - ссылка на страничку авито, где есть товары и прокрутка страниц, чтобы можно было добавить
    "?p=<номер страницы> в конец url
    max_accounts - максимальное количество аккаунтов, которое вытаскивает функция search '''
    global proxer
    global accounts
    
    # Номер страницы на avito
    page = start_page

    ## Контроль количества потоков
#     for i in range(54):
    for i in range(3, 54, 3):
        t = Thread(target=get_acc_from_product, args=(list(range(i-3, i)),))
        t.start()
        
    while True:
        get_accounts_from_page(url=(url + r'?p={}').format(page))
        page += 1

In [ ]:
def parse():
    global accounts
    global visited_accounts
    global visited_accounts_lock
    browser = get_new_browser()
    
    while True:
        url = pop_account()
        browser = get_url(browser, url)
        
        name = browser.find_element_by_class_name('profile-public-name-edUBF')
        data = browser.find_elements_by_class_name('profile-public-summary-item-wFbyw')
        prod = browser.find_elements_by_class_name('styles-date-2vN1V')
        cost = browser.find_elements_by_class_name('styles-price-1RC3V')
        i = 0
        l = len(prod) - 1
        l2 = l
        l3 = l2
        while(l != len(prod)):
            l3 = l2
            l = len(prod)
            l2 = l
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            random_sleep()
            prod = browser.find_elements_by_class_name('styles-date-2vN1V')
            cost = browser.find_elements_by_class_name('styles-price-1RC3V')
            try:
                element = browser.find_element_by_class_name('styles-button-10MP4')
                element.click()
                l = l - 1
                random_sleep()
            except Exception as e:
#                 print(e)
                pass
            if(l3 == len(prod)):
                l = len(prod)
        person_info = {
            'url': url,
            'name': name.text,
            'data': list(map(lambda x: x.text, data)),
            'prod_data_active': list(map(lambda x: x.text, prod)),
            'prod_cost_active': list(map(lambda x: x.text, cost))
        }
        c = browser.find_element_by_partial_link_text('Завершённые')
        c.click()
        random_sleep()
        prodold = browser.find_elements_by_class_name('styles-date-2vN1V')
        costold = browser.find_elements_by_class_name('styles-price-1RC3V')
        l = len(prodold) - 1
        l2 = l
        l3 = l2
        if browser.page_source.find('Нет завершённых объявлений') < 0:
            while(l != len(prodold)):
                l3 = l2
                l = len(prodold)
                l2 = l
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                random_sleep()
                prodold = browser.find_elements_by_class_name('styles-date-2vN1V')
                costold = browser.find_elements_by_class_name('styles-price-1RC3V')
                try:
                    element = browser.find_element_by_class_name('styles-button-10MP4')
                    element.click()
                    l = l - 1
                    random_sleep()
                except:
                    pass
                if(l3 == len(prod)):
                    l = len(prodold)

        person_info['prod_data_finished'] = list(map(lambda x: x.text, prodold))
        person_info['prod_cost_finished'] = list(map(lambda x: x.text, costold))
        
        
        
        add_data(person_info)


def strtodays(s):
    months = ['января', 'февраля', 'марта', 'апреля', 'мая', 'июня',
                  'июля', 'августа', 'сентября', 'октября', 'ноября', 'декабря'] 
    
    if s.lower().find('сегодня') >= 0:
        t = date.today()
        return date(t.year, t.month, t.day).toordinal()
    elif s.lower().find('вчера') >= 0:
        t = date.today() - timedelta(days=1)
        return date(t.year, t.month, t.day).toordinal()
    
    m = re.match(r'.*(\d\d\d\d).*', s)
    year = date.today().year if m is None else int(m.group(1))
    month = 0
    for i in range(12):
        if s.find(months[i]) >= 0:
            month = i + 1
            break
    day = int(re.match(r'(\d*) .*', s).groups(1)[0])

    return date(year, month, day).toordinal()


def normalize(account):
    result = dict()
    result['name'] = account['name'].lower()
    result['person_status'] = account['data'][0].lower()
    result['url'] = account['url']
    
    start_date = account['data'][1]
    if start_date.startswith('На Авито с '):
        months = ['января', 'февраля', 'марта', 'апреля', 'мая', 'июня',
                  'июля', 'августа', 'сентября', 'октября', 'ноября', 'декабря'] 
        month = 0
        for i in range(12):
            if start_date.find(months[i]) >= 0:
                month = i + 1
                break
        result['start_date'] = str(date(int(re.match('.*(\d\d\d\d).*', start_date).group(1)), month, 1).toordinal())
    else:
        result['start_date'] = ''
        
    result['answer_time'] = account['data'][3]
    result['first_sold_time'] = '' if len(account['prod_data_finished']) == 0 else str(strtodays(account['prod_data_finished'][-1]))
    result['n_active'] = str(len(account['prod_data_active']))
    result['n_finished'] = str(len(account['prod_data_finished']))
    result['active_finished_ratio'] = str((len(account['prod_data_active']) + 1) / (len(account['prod_data_finished']) + 1))
    
    active_costs = []
    for cost in account['prod_cost_active']:
        active_costs.append(int(re.match('(\d*).*', cost.replace(' ', '')).group(1)))
        
    finished_costs = []
    for cost in account['prod_cost_finished']:
        finished_costs.append(int(re.match('(\d*).*', cost.replace(' ', '')).group(1)))
        
    result['min_active_cost'] = str(min(active_costs)) if len(active_costs) > 0 else ''
    result['max_active_cost'] = str(max(active_costs)) if len(active_costs) > 0 else ''
    result['mean_active_cost'] = str(sum(active_costs) / len(active_costs)) if len(active_costs) > 0 else ''
    
    result['min_finished_cost'] = str(min(finished_costs)) if len(finished_costs) > 0 else ''
    result['max_finished_cost'] = str(max(finished_costs)) if len(finished_costs) > 0 else ''
    result['mean_finished_cost'] = str(sum(finished_costs) / len(finished_costs)) if len(finished_costs) > 0 else ''
    return result

##########################################################################
#####################CONFIGURATIONS#######################################
url = r'https://www.avito.ru/rossiya/lichnye_veschi'
HASH_MOD = [2, 3, 4]
num_of_parsers = 5
##########################################################################
##########################################################################

products = [None] * 54
accounts = []

proxer = get_proxy_gen()
data = {'name':[], 'min_active_cost':[], 'max_active_cost':[], 'mean_active_cost':[], 'min_finished_cost':[],
        'max_finished_cost':[], 'mean_finished_cost':[], 'answer_time':[], 'first_sold_time':[], 'n_active':[],
        'n_finished':[], 'active_finished_ratio':[], 'person_status':[], 'url':[], 'start_date':[]}

accounts_lock = Lock()
proxer_lock = Lock()
data_lock = Lock()

visited_products = get_visited_products()
visited_accounts = get_visited_accounts()

visited_products_lock = Lock()
visited_accounts_lock = Lock()

df = pd.DataFrame(data)

while True:
    try:
        searcher = Thread(target=search, args=(url, ))
        searcher.start()
        
        for _ in range(num_of_parsers):
            parser = Thread(target=parse)
            parser.start()
        
        while True:
            sleep(300)
            save_data()
    except:
        pass


got account: https://www.avito.ru/user/01071faa44efa2710511d74c3ffc7887/profile
start to parse account: https://www.avito.ru/user/01071faa44efa2710511d74c3ffc7887/profile
got account: https://www.avito.ru/user/6fa9385653f303bd0a7a2e8a2724d2a2/profile
got account: https://www.avito.ru/user/6d0b119fb6c8ca97601117e96e0e1cae/profile
start to parse account:start to parse account: https://www.avito.ru/user/6d0b119fb6c8ca97601117e96e0e1cae/profile
 https://www.avito.ru/user/6fa9385653f303bd0a7a2e8a2724d2a2/profile
got account: https://www.avito.ru/user/f04a643263948ad0918ee44037a54e0d/profile
start to parse account: https://www.avito.ru/user/f04a643263948ad0918ee44037a54e0d/profile
got account: https://www.avito.ru/user/01d4fb116ad472968fb11cb76b8f4a6f/profile
start to parse account: https://www.avito.ru/user/01d4fb116ad472968fb11cb76b8f4a6f/profile
got account: https://www.avito.ru/user/d295d282a00e07999dbc4a5f78b5ac65/profile
got account: https://www.avito.ru/user/a5b1dc29fc8404b4ade3d0b4c0